## Collaborative Filtering

In [17]:
# import pandas
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

We will create an recommender engine based on Item Based Collaborative Filtering (IBCF) which searches for the most similar books based on the user ratings. We can download the data from [here](https://drive.google.com/file/d/1WvTmAfO09TCX7xp7uu06__ziic7JnrL5/view?usp=sharing).

In [2]:
book_ratings = pd.read_csv('BX-Book-Ratings.csv',sep=";", encoding="latin")
books = pd.read_csv('BX-Books.csv',sep=";", encoding="latin", error_bad_lines=False)

C:\Users\Nik\anaconda3\envs\lhl\lib\site-packages\IPython\core\interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933

* Explore both datasets

In [3]:
book_ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [4]:
book_ratings.isna().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [5]:
book_ratings['User-ID'].nunique()

105283

In [6]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [7]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [8]:
books.dropna(inplace=True)

In [9]:
books.isna().sum()

ISBN                   0
Book-Title             0
Book-Author            0
Year-Of-Publication    0
Publisher              0
Image-URL-S            0
Image-URL-M            0
Image-URL-L            0
dtype: int64

* create dataframe with name 'df_book_features' from book_ratings that have `ISBN` as index, `User-ID` as columns and values are `Book-Rating`.
    - The data are quite big so it's OK to use a sample only in case your PC has limited RAM.


In [10]:
df_temp = pd.merge(book_ratings, books, on = 'ISBN')
df_temp

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
...,...,...,...,...,...,...,...,...,...,...
1031124,276688,0517145553,0,Mostly Harmless,Douglas Adams,1995,Random House Value Pub,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...,http://images.amazon.com/images/P/0517145553.0...
1031125,276688,1575660792,7,Gray Matter,Shirley Kennett,1996,Kensington Publishing Corporation,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...,http://images.amazon.com/images/P/1575660792.0...
1031126,276690,0590907301,0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1997,Apple,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...,http://images.amazon.com/images/P/0590907301.0...
1031127,276704,0679752714,0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1997,Vintage Books USA,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...,http://images.amazon.com/images/P/0679752714.0...


In [11]:
df_book_features = df_temp.sample(100000).pivot_table(index='ISBN', columns='User-ID', values='Book-Rating')
df_book_features.head()

User-ID,8,10,14,19,22,26,36,51,67,85,...,278663,278672,278694,278714,278723,278774,278815,278843,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000104687X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001047973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001053736,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001360469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0001837397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_book_features.mean(axis=1)

ISBN
000104687X     6.0
0001047973     9.0
0001053736     5.0
0001360469    10.0
0001837397     0.0
              ... 
B00013AFZQ     0.0
B00016560C     0.0
B00019U4II     8.0
B0001FZGPI     0.0
B0001I1KOG    10.0
Length: 58829, dtype: float64

In [36]:
#df_book_features2 = df_book_features.fillna(df_book_features.mean(axis=0))
df_book_features2 = df_book_features.fillna(0)
df_book_features2.head(5)

User-ID,8,10,14,19,22,26,36,51,67,85,...,278663,278672,278694,278714,278723,278774,278815,278843,278851,278854
ISBN,,,,,,,,,,,,,,,,,,,,,
000104687X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001047973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001053736,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001360469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001837397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


* create the instance of the NearestNeighbors class

In [49]:
nn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)

* fit the NearestNeighbors using'df_book_features'

In [50]:
nn.fit(df_book_features2)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1)

* create function that returns top 5 most similar movies (according to KNN model) for selected ISBN
    * the input will be Book-Title from the DataFrame books 
    * the output will be the Book-Titles of the top 5 most similar books.
    * for every book in the top 5 most similar books, print also the distance from the selected book (ISBN we chose as input to the function)

In [38]:
#attempt = df_book_features2.iloc[0,:]
attempt = df_book_features2.loc[['000104687X']]

In [39]:
temp_arr = np.array([attempt])
reshaped_array = temp_arr.reshape(1, -1)
nn.kneighbors(reshaped_array)

(array([[832.1241925 , 832.4925152 , 832.50978261, 832.51117674,
         832.51220225]]),
 array([[49679, 10075, 28613, 21257, 23393]], dtype=int64))

In [37]:
recs = nn.kneighbors(np.array(df_book_features2.loc[['000104687X']]).reshape(1, -1), n_neighbors=5, return_distance=True)
recs

(array([[832.1241925 , 832.4925152 , 832.50978261, 832.51117674,
         832.51220225]]),
 array([[49679, 10075, 28613, 21257, 23393]], dtype=int64))

In [23]:
df_book_features2.iloc[23266,:].name

'0446531332'

In [24]:
books[books['ISBN'] == '0001052039']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
199111,0001052039,Dark Spectre,Michael Dibdin,1997,HarperCollins Publishers,http://images.amazon.com/images/P/0001052039.0...,http://images.amazon.com/images/P/0001052039.0...,http://images.amazon.com/images/P/0001052039.0...


In [25]:
books[books['ISBN'] == '0446604658']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
1983,0446604658,Lucky You,Carl Hiaasen,1998,Warner Vision,http://images.amazon.com/images/P/0446604658.0...,http://images.amazon.com/images/P/0446604658.0...,http://images.amazon.com/images/P/0446604658.0...


In [33]:
books[books['ISBN'] == '0001047973']

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
12764,0001047973,Brave New World,Aldous Huxley,1999,Trafalgar Square Publishing,http://images.amazon.com/images/P/0001047973.0...,http://images.amazon.com/images/P/0001047973.0...,http://images.amazon.com/images/P/0001047973.0...


In [27]:
books[books['Book-Title'] == 'Dark Spectre']['ISBN'].values[0]

'0571177530'

In [29]:
#df_book_features2.loc[['000184251X']]

In [30]:
# Function will return an error if multiple ISBNs are found and one is not in the sampled dataframe
books[books['Book-Title'] == 'Dark Spectre']['ISBN'].values

array(['0571177530', '0001052039'], dtype=object)

In [51]:
def similar_books(title):
    temp_ISBN = books[books['Book-Title'] == title]['ISBN'].values[0]
    temp_row = df_book_features2.loc[[temp_ISBN]]
    temp_arr = np.array([temp_row])
    reshaped_array = temp_arr.reshape(1, -1)
    
    for i in range(0,5):
        lookup_idx = nn.kneighbors(reshaped_array)[1][0][i]
        lookup_ISBN = df_book_features2.iloc[lookup_idx,:].name
        print(books[books['ISBN'] == lookup_ISBN]['Book-Title'].values[0])
        
    return

In [52]:
similar_books('Brave New World')

Brave New World
New World Order
Esther's Gift: A Mitford Christmas Story (Christmas in Mitford Gift)
The New Baby (Usborne First Experiences)
Games.... (Puffin Books)


In [53]:
similar_books('The Shining')

The Shining
A Cup of Light
What Should I Do? (All About You)
Boys Against Girls
Round Ireland In Low Gear


In [44]:
def make_recommendations(book_title):
    """
    make top 5 book recommendations
    Parameters
    ----------
    fav_book: str, name of user input book
    """
    # get isbn
    ISBN = books[books['Book-Title'] == book_title]['ISBN'].values[0]
    # get recommendations
    recs = nn.kneighbors(np.array(df_book_features2.loc[[ISBN]]).reshape(1, -1), n_neighbors=5, return_distance=False)
    # print results
    recommends = []
    for i in recs:
        recommends.append(df_book_features2.iloc[i].index)
    rec_isbn = []
    for i in range(0,len(recommends[0])):
        rec_isbn.append(recommends[0][i])
    for i in range(0,len(rec_isbn)):
        print(books[books['ISBN'] == rec_isbn[i]]['Book-Title'].values[0])
        
    return

In [48]:
make_recommendations('The Alchemist')

Wild Animus
How to Make an American Quilt
High Society
Circle of Friends
Lucky You


* Apply the function to book of your choice